# Boolean Calculator: Top-down Parser


## Translation scheme

|Translation scheme                                                                                   |
|-----------------------------------------------------------------------------------------------------|
| entry -> `print` exprOR `;`{write('The result is {exprOR.s} ;')}                                    |
| entry -> asign `;`                                                                                    |
| asign -> `ID` `=` exprOR   { Table\[ID\] = exprOR.s }                                                 |
| exprOR -> exprAND { exprOR'.h = exprAND.s} exprOR' { exprOR.s = eprOR' }                            |
| exprOR' -> `or` exprAND { exprOR_1'.h = exprOR.h or exprAND.s } exprOR_1' { exprOR'.s = exprOR_1.s }|
| exprOR' -> e { exprOR'.s = exprOR'.h }                                                              |
| exprAND -> boolean { exprAND'.h = boolean.s } exprAND' { exprAND.s = exprAND'.s }                   |
| exprAND' -> `and` boolean { exprAND_1.h = exprAND'.h and boolean.s} exprAND_1'{exprAND'.s = exprAND_1'.s } |
| exprAND' -> e { exprAND'.s = exprAND'.h }                                                           |
| boolean -> not boolean_1 { boolean.s = not boolean_1.s }                                            |
| boolean -> `CBOOLEAN` { boolean.s = CBOOLEAN.lexval }                                               |
| boolean -> `ID` { boolean.s = table\[ID.lexval\] }                                                    |
| boolean -> `(` exprOR `)` { boolean.s = exprOR.s }                                                  |


## First and Follow sets

| Sintactic rules                    |
|------------------------------------|
| entry -> `print` exprOR `;`        |
| entry -> asign `;`                   |
| asign -> `ID` `=` exprOR           |
| exprOR -> exprAND exprOR'          |
| exprOR' -> `or` exprAND exprOR'    |
| exprOR' -> e                       |
| exprAND -> boolean exprAND'        |
| exprAND' -> `and` boolean exprAND' |
| exprAND' -> e                      |
| boolean -> not boolean             |
| boolean -> `CBOOLEAN`              |
| boolean -> `ID`                    |
| boolean -> `(` exprOR `)`          |


### First set

```
First(entry) = { Print , ID}
First(asign) = { ID }
First(ExprOR) = { not, CBOOLEAN, ID, '(' }
First(ExprOR') = { 'or', e }
First(ExprAND) = { not, CBOOLEAN, ID, '(' }
First(ExprAND') = { and, e }
First(Boolean) = {not, CBOOLEAN, ID, '(' }
```

### Follow set
```
Follow(entry) = {$}
Follow(asign) = {';'}
Follow(exprOR) = Follow(asign) U {')'} = { ';', ')' }
Follow(exprOR') = Follow(exprOR) U Follow(exprOR')  = { ';', ')' }
Follow(exprAND) = First(exprOR') - {e} U Follow(exprOR') = {or, ';', ')' }
Follow(exprAND') = Follow(exprAND) U Follow(exprAND') = {or, ';', ')' }
Follow(boolean) = First(exprAND') - {e} U  Follow(exprAND) U Follow(boolean) U { ')' } = {and, or, ';', ')'}
```

# Necessary libraries

In [74]:
!pip install sly

You should consider upgrading via the 'c:\users\los_g\onedrive\documentos\pl\pl-2020\prácticas\boolean-calculator-sintax-direct-translation\.env\scripts\python.exe -m pip install --upgrade pip' command.


# Lexer implementation

In [75]:
from sly import Lexer

class BooleanCalcLexer(Lexer):
    tokens = {ID, CBOOLEAN, ASG, NOT, OR, AND, PRINT}
    ignore = ' \t'
    literals = {';', '(', ')'}

    # Regular expression rules for tokens
    ID = r'[a-zA-Z_][a-zA-Z0-9]*'
    ID['not'] = NOT
    ID['and'] = AND
    ID['or'] = OR
    ID['false'] = CBOOLEAN
    ID['true'] = CBOOLEAN
    ID['print'] = PRINT
    ASG = r':='

    @_(r'\n+')
    def newline(self, t):
        self.lineno += t.value.count('\n')
    
    def error(self, t):
        print("line {}: Bad character {}".format(self.lineno, t.value[0]))
        self.index += 1


In [76]:
    if __name__ == '__main__':
        data = 'z := true or not (x and not y);'
        lexer = BooleanCalcLexer()
        for token in lexer.tokenize(data):
            print(token)

Token(type='ID', value='z', lineno=1, index=0)
Token(type='ASG', value=':=', lineno=1, index=2)
Token(type='CBOOLEAN', value='true', lineno=1, index=5)
Token(type='OR', value='or', lineno=1, index=10)
Token(type='NOT', value='not', lineno=1, index=13)
Token(type='(', value='(', lineno=1, index=17)
Token(type='ID', value='x', lineno=1, index=18)
Token(type='AND', value='and', lineno=1, index=20)
Token(type='NOT', value='not', lineno=1, index=24)
Token(type='ID', value='y', lineno=1, index=28)
Token(type=')', value=')', lineno=1, index=29)
Token(type=';', value=';', lineno=1, index=30)


# Parser implementation

In [77]:
table = {}
ta = None
ind = 0
tokens = None
tokenlist = None


In [78]:
def entry():
    global ta, ind, tokens
    if ta.type == 'PRINT':
        cuadra('PRINT')
        exprOR_s = exprOR()
        cuadra(';')
        print('The result is {}'.format(exprOR_s))
    elif ta.type == 'ID':
        asign()
        cuadra(';')
        if ind < len(tokenlist): #fin de la entrada
            yyerror("in entry")
    else:
        yyerror("in entry")

def asign():
    global ta, tokens
    if ta.type == 'ID':
        IDlexval = ta.value
        cuadra('ID')
        cuadra('ASG')
        exprOR_s = exprOR()
        table[IDlexval] = exprOR_s

    else:
        yyerror("in asign")

def exprOR():
    global ta, tokens
    if ta.type == 'NOT' or ta.type == 'CBOOLEAN' or ta.type == 'ID' or ta.type == '(':
        exprAND_s = exprAND()
        exprOR1_s = exprOR1(exprAND_s)
        return exprOR1_s
    else:
        yyerror("in exprOR")

def exprOR1(exprOR1_h):
    global ta, tokens
    if ta.type == 'OR':
        cuadra('OR')
        print(ta)
        exprOR11_h = exprOR1_h | exprAND()
        print(exprOR11_h)
        exprOR11_s = exprOR1(exprOR11_h)
        return exprOR11_s
    else:
        if ta.value != ';' and ta.value != ')':
            yyerror("in exprOR1")
        else:
            exprOR1_s = exprOR1_h
            return exprOR1_s

def exprAND():
    global ta, tokens
    if ta.type == 'NOT' or ta.type == 'CBOOLEAN' or ta.type == 'ID' or ta.type == '(':
        boolean_s = boolean()
        exprAND1_s = exprAND1(boolean_s)
        return exprAND1_s
    else:
        yyerror("in exprAND")

def exprAND1(exprAND1_h):
    global ta, tokens
    if ta.type == 'AND':
        cuadra('AND')
        boolean_s = boolean()
        exprAND1_h = boolean_s & exprAND1_h
        exprAND11_s = exprAND1(exprAND1_h)
        return exprAND11_s

    elif ta.type != 'OR' and ta.type != ';' and ta.type != ')':
        yyerror('in exprAND1')
    else:
        exprAND1_s = exprAND1_h
        return exprAND1_s



def boolean():
    global ta, tokens
    boolean_s = None

    if ta.type == 'NOT':
        cuadra('NOT')
        boolean_s = not boolean()
    elif ta.type == 'CBOOLEAN':
        boolean_s =  True if ta.value == 'true' else False
        cuadra('CBOOLEAN')
    elif ta.type == 'ID':
        boolean_s = table[ta.value]
        cuadra('ID')
    elif ta.type == '(':
        cuadra('(')
        boolean_s = exprOR()
        cuadra(')')
    else:
        yyerror('in boolean')
    
    return boolean_s

# Aux functions

In [79]:
def yyerror(msg):
    print("Sintax error", msg)

def cuadra(obj):
    global ta, ind, tokenlist
    if ta.type == obj:
        # print("squaring ta = {}".format(ta.value))
        ind += 1
        if ind < len(tokenlist):
            ta = tokenlist[ind]
            # print("==> new ta = ", ta.value)
    else:
        yyerror("in cuadra")

In [91]:
def testing(sentence):
    global ta, ind, tokens
    lexer = BooleanCalcLexer()
    tokens = BooleanCalcLexer.tokens
    tokens == sorted(tokens)
    print('Starting the program')
    
    if data:
        tokenlist = list(lexer.tokenize(sentence))
        ta = tokenlist[ind]
        entry()
    ind = 0
    print("Program finished: resulting variables")
    print(table)


In [92]:
sentences = ['x:=true;', 'y:=false', 'z := true or not (x and not y);']
for sentence in sentences:
    testing(sentence)

Starting the program
Program finished: resulting variables
{'a': True, 'x': True}
Starting the program
Program finished: resulting variables
{'a': True, 'x': True, 'y': True}
Starting the program
Program finished: resulting variables
{'a': True, 'x': True, 'y': True, 'z': True}
